In [30]:
import torch 
import matplotlib.pyplot as plt
import random
import torch.nn.functional as F
import torch.nn as nn

%matplotlib inline

In [3]:
#load  dataset to get list of strings  
words=open('names.txt','r').read().splitlines()

In [4]:
#little info about the data 
print('total number of words in a dataset',len(words))
print('shortest word in the dataset',min(len(w)for w in words))
print('longest word in the dataset',max(len(w)for w in words))

total number of words in a dataset 32033
shortest word in the dataset 2
longest word in the dataset 15


In [5]:
#create dictionary
chars=sorted(list(set(''.join(words))))
#string to integer
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
#integer to string 
itos={i:s for s,i in stoi.items()}


E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see.

    train: 2.2833642959594727

    dev loss 2.277104616165161

    test loss 2.277104616165161

so the model performs almost the same on all 3 dataset's 


In [6]:
def split(words):
    xs,ys=[],[]
    for w in words:
        chs=['.']+list(w)+['.']
        for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
            ix1=stoi[ch1]
            ix2=stoi[ch2]
            ix3=stoi[ch3]
            xs.append([ix1,ix2])
            ys.append(ix3)

    xs=torch.tensor(xs)
    ys=torch.tensor(ys)
    return xs,ys

In [7]:
random.shuffle(words)

#split 80% train ,10% dev,10% test 
train_size=int(0.8*len(words))
dev_size=int(0.9*len(words))

#train,dev,test datasets 
x_train,x_test=split(words[:train_size])
d_train,d_test=split(words[:dev_size])
t_train,t_test=split(words[dev_size:])


E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?  

so after 100 epochs we did get a little improvement 

    -bigram loss after 100 epochs: 2.4884581565856934

    -trigram loss after 100 epochs: 2.2833642959594727


i reduce the learning_rate to be 25,even that its a huge learning rate,probably adding a l1 reqularization would help,and for sure having a activation func(RELU). 


In [21]:
g = torch.Generator().manual_seed(2147483647)
#since we now using 2 char to predict the 3rd one we need to have [27*2,27]  
W_trigram=torch.rand((27*2,27),generator=g,requires_grad=True)
l2_rate=0.01
learning_rate=40

for k in range(100):
    #turn two char into one hot encoded vectors shape [N,2,27] cast to float 
    xenc=F.one_hot(x_train,num_classes=27).float()
    #view returns a new tensor with the same data diff shape [N,27*2] 
    logits=xenc.view(-1,27*2)@W_trigram
    #softmax 
    counts=logits.exp()
    probs=counts/counts.sum(1,keepdims=True)
    #negative log likelihood + l2 regularization
    loss= -probs[torch.arange(x_test.shape[0]), x_test].log().mean() + l2_rate * (W_trigram**2).mean()
    print(loss.item())

    #backward pass 
    W_trigram.grad=None
    loss.backward()

    #gradient descent
    W_trigram.data+=-learning_rate*W_trigram.grad



3.4314181804656982
2.937809467315674
2.7350199222564697
2.6303911209106445
2.564183235168457
2.5189216136932373
2.4860715866088867
2.4611103534698486
2.4414167404174805
2.4253995418548584
2.4120519161224365
2.4007134437561035
2.390932559967041
2.3823912143707275
2.37485671043396
2.3681538105010986
2.362147569656372
2.356731653213501
2.351821184158325
2.3473458290100098
2.343250036239624
2.339484930038452
2.3360114097595215
2.3327958583831787
2.3298094272613525
2.3270280361175537
2.3244307041168213
2.3219985961914062
2.319716453552246
2.3175699710845947
2.315547466278076
2.3136379718780518
2.3118324279785156
2.310121774673462
2.3084986209869385
2.3069570064544678
2.305490255355835
2.3040926456451416
2.302760124206543
2.3014872074127197
2.3002710342407227
2.299107313156128
2.29799222946167
2.2969233989715576
2.295897960662842
2.294912576675415
2.2939655780792236
2.2930550575256348
2.292178153991699
2.2913334369659424
2.2905187606811523
2.2897331714630127
2.2889745235443115
2.288241863250

In [22]:
#evaluate func for  dev,test datasets  
def evaluate(features,labels):
    xenc=F.one_hot(features,num_classes=27).float()
    #we can think that this is like the first layer 
    logits = xenc.view(-1, 27*2) @ W_trigram 
    #softmax
    counts=logits.exp()
    probs=counts/counts.sum(1,keepdims=True)
    #negative log likelihood + l2 regularization
    loss= -probs[torch.arange(features.shape[0]), labels].log().mean() +  l2_rate * (W_trigram**2).mean()
    return loss.item()

In [23]:
dev_loss=evaluate(d_train,d_test)
test_loss=evaluate(d_train,d_test)
print('dev loss',dev_loss)
print('test loss',test_loss)

dev loss 2.2701611518859863
test loss 2.2701611518859863


E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

trying to set the l2_learning_rate>0.01 ex:0.1 is actually giving  worse results.

the learning rate when we set it >50 is way to high and anything below 40 i think its a small learning rate(lol) so probably adding here a learning rate decay would help to be able to decrease the learning_rate over epochs.

loss archived given in EO1 desc.


     

E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

So the reason is mainly efficiency since computing the derivatives of Softmax and Categorical cross entropy separate is wasteful it can be simplified a lot to get more efficient function to compute the gradients of both functions,and less code :)  



In [45]:
g = torch.Generator().manual_seed(2147483647)
#since we now using 2 char to predict the 3rd one we need to have [27*2,27]  
W_trigram=torch.rand((27*2,27),generator=g,requires_grad=True)
l2_rate=0.01
l1_rate=0.01
learning_rate=60
decay=0.01
for k in range(100):
    #turn two char into one hot encoded vectors shape [N,2,27] cast to float 
    xenc=F.one_hot(x_train,num_classes=27).float()
    #view returns a new tensor with the same data diff shape [N,27*2] 
    logits=xenc.view(-1,27*2)@W_trigram
    
    #relu nonlinearity
    relu=nn.ReLU()
    relu_out=relu(logits)

    #with categorical cross entropy
    loss = F.cross_entropy(relu_out, x_test)
    loss+l1_rate*(W_trigram.sum())
    loss+=l2_rate*(W_trigram**2).mean()
    print(loss.item())

    #backward pass 
    W_trigram.grad=None
    loss.backward()

    #add learning rate decay:
    current_learning_rate=learning_rate*(1/(1+decay*k))

    #gradient descent
    W_trigram.data+=-current_learning_rate*W_trigram.grad

3.4314181804656982
2.7966766357421875
2.6260831356048584
2.540867805480957
2.495635986328125
2.461956262588501
2.442763566970825
2.421114921569824
2.409801483154297
2.3941047191619873
2.3860316276550293
2.3745193481445312
2.3681862354278564
2.359614372253418
2.354408025741577
2.3478848934173584
2.3434760570526123
2.3384199142456055
2.334667444229126
2.3306963443756104
2.327516555786133
2.3243472576141357
2.321659803390503
2.3190925121307373
2.3168270587921143
2.3147048950195312
2.3127834796905518
2.3109941482543945
2.3093395233154297
2.307792901992798
2.3063464164733887
2.304987907409668
2.30370831489563
2.302500009536743
2.3013558387756348
2.300269603729248
2.2992360591888428
2.298252582550049
2.297316312789917
2.296422004699707
2.2955665588378906
2.2947490215301514
2.2939679622650146
2.293219566345215
2.2925028800964355
2.2918152809143066
2.2911555767059326
2.2905213832855225
2.289910316467285
2.2893216609954834
2.288754463195801
2.2882070541381836
2.2876780033111572
2.28716754913330

E06: meta-exercise! Think of a fun/interesting exercise and complete it.	

	-adding learning rate decay 
	
	-adding a relu non-linearity   

	-adding l1 regularization 
	
code above


In [58]:
#check whats wrong here in sampling 
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out=[]
    ix=0
    while True:
        #linear layer
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27*2).float()
        #matrix multiplication
        logits=xenc@W_trigram
        #relu
        relu=nn.ReLU()
        relu_out=relu(logits)
        #softmax 
        softmax=nn.Softmax(dim=1)
        p=softmax(relu_out)

        #sample from multinomial distribution 
        ix=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        out.append(itos[ix])
        if ix==0:
            break
        print(''.join(out))


o
ot
otq
otqa
x
xx
xxa
xxan
xxann
xxannl
xxannli
xxannlix
xxannlixp
xxannlixpw
xxannlixpwt
xxannlixpwtz
xxannlixpwtzk
xxannlixpwtzki
xxannlixpwtzkis
xxannlixpwtzkisa
xxannlixpwtzkisal
xxannlixpwtzkisalt
xxannlixpwtzkisalti
xxannlixpwtzkisaltii
xxannlixpwtzkisaltiio
h
ht
hta
htae
a
an
ani
anii
aniiw
aniiwm
aniiwmr
aniiwmrz
aniiwmrzc
aniiwmrzcs
aniiwmrzcsm
aniiwmrzcsmi
r
rb
rbe
rber
rberh
rberhn
rberhna
rberhnah
rberhnahc
rberhnahce
rberhnahcee
rberhnahceec
rberhnahceect
rberhnahceecti
rberhnahceectia
rberhnahceectiah
rberhnahceectiahe
rberhnahceectiaheh
rberhnahceectiahehq
rberhnahceectiahehqq
rberhnahceectiahehqqn
rberhnahceectiahehqqne
rberhnahceectiahehqqnet
rberhnahceectiahehqqnetl
rberhnahceectiahehqqnetlg
rberhnahceectiahehqqnetlgo
rberhnahceectiahehqqnetlgoc
rberhnahceectiahehqqnetlgoce
rberhnahceectiahehqqnetlgocee
rberhnahceectiahehqqnetlgoceey
rberhnahceectiahehqqnetlgoceeyt
rberhnahceectiahehqqnetlgoceeyti
rberhnahceectiahehqqnetlgoceeytir
